<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Mar 30, 2022</h5>

# Set up and requirements

In [ ]:
# Code to try fix the upload video panoptes libmagic
# !pip install panoptes-client
# !pip uninstall python-magic
# !pip install python-magic==0.4.25
# !pip install python-magic-bin
# !pip install libmagic==1.0
# !pip uninstall python-magic
# !pip install python-libmagic
# !pip install git+https://github.com/julian-r/python-magic.git

In [ ]:
!git clone --recurse-submodules -b dev https://github.com/ocean-data-factory-sweden/koster_data_management.git
!pip install -r koster_data_management/requirements.txt

import site
# Deal with github installed packages
site.main()

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

os.chdir("koster_data_management/tutorials")
sys.path.append('..')


# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.movie_utils as m_utils
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils

print("Packages loaded successfully")

### Choose your project

In [3]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU', 'Medins'), value='…

In [4]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and zoo project

In [5]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

Enter the key id for the aws server··········
Enter the secret access key for the aws server··········


None/sites_buv_doc.csv: 100%|██████████| 145k/145k [00:01<00:00, 113kB/s]
None/movies_buv_doc.csv: 100%|██████████| 14.2k/14.2k [00:00<00:00, 31.7kB/s]
None/species_buv_doc.csv: 100%|██████████| 7.53k/7.53k [00:00<00:00, 17.5kB/s]
None/surveys_buv_doc.csv: 100%|██████████| 1.78k/1.78k [00:00<00:00, 4.08kB/s]
None/choices_buv.csv: 100%|██████████| 3.54k/3.54k [00:00<00:00, 7.82kB/s]


Updated sites
Updated movies
Updated species


In [6]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

Enter your Zooniverse user··········
Enter your Zooniverse password··········


# Retrieve info about zooniverse clips

In [7]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project = project, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Updated subjects
The database has a total of 978 frame subjects and 4993 clip subjects have been updated


# Retrieve info about movies hosted in the server

In [8]:
# Store info about the movies available in the server
available_movies_df = t3.retrieve_movie_info_from_server(project = project,
                                                         db_info_dict = db_info_dict)

### Select the movie you want to upload to Zooniverse

In [9]:
movie_i = t3.movie_to_upload(available_movies_df = available_movies_df)

Dropdown(description='Movie to upload:', layout=Layout(width='50%'), options=('Te Oneroa BayKAP01_2021', 'Hole…

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [10]:
# Check movie hasn't been uploaded to Zooniverse
t3.check_movie_uploaded(movie_i = movie_i.value,
                        db_info_dict = db_info_dict)

TUH_026_1_09_2020 has not been uploaded to Zooniverse yet


Have a quick look at the movie selected

In [11]:
import ipywidgets as widgets
from IPython.display import HTML
from kso_utils.t4_utils import get_movie_url
def preview_movie(project, db_info_dict, available_movies_df, movie_i):

  # Add the key of the movie of interest
  movie_selected = available_movies_df[available_movies_df["filename"]==movie_i.value].reset_index(drop=True)

  # Make sure only one movie is selected
  if len(movie_selected.index)>1:
    print("There are several movies with the same filename. This should be fixed!")
  
  else:
    
    # Generate ctemporary path to the movie select
    movie_selected["movie_path"] = get_movie_url(project, db_info_dict, movie_selected["fpath"].values[0])

    return HTML(f"""<video src={movie_selected["movie_path"].values[0]} width=800 controls/>""")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2850816/45929032 bytes (6.2%)6496256/45929032 bytes (14.1%)10051584/45929032 bytes (21.9%)13467648/45929032 bytes (29.3%)16949248/45929032 bytes (36.9%)20480000/45929032 bytes (44.6%)24223744/45929032 bytes (52.7%)27648000/45929032 bytes (60.2%)31244288/45929032 bytes (68.0%)35012608/45929032 bytes (76.2%)38739968/45929032 bytes (84.3%)42344448/45929032 bytes (92.2%)

In [12]:
preview_movie(project,db_info_dict,available_movies_df,movie_i)

### Specify the number of clips and clip length

In [13]:
clip_selection = t3.select_clip_n_len(movie_i = movie_i.value,
                                      db_info_dict = db_info_dict)

interactive(children=(Dropdown(description='Length of clips:', options=(10, 5), style=DescriptionStyle(descrip…

### Review the clips that will be created

In [14]:
t3.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

You are about to create 1 clips from TUH_026_1_09_2020
starting at 0:06:15 and ending at 0:06:25


### Create the clips

In [20]:
clips_to_upload_df = t3.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value,
                                     db_info_dict = db_info_dict,
                                     clip_selection = clip_selection,  
                                     project = project)

TUH_026_1_09_2020.MP4: 100%|██████████| 13.2G/13.2G [02:15<00:00, 97.0MB/s]
100%|██████████| 1/1 [00:00<00:00,  1.08it/s]

clips extracted successfully


Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Review clips

In [21]:
# Review the size of the modified clips
t3.check_clip_size(clips_df = clips_to_upload_df)

Clips are too large (over 8 MB) to be uploaded to Zooniverse. Compress them!


,File_path,Size
0,TUH_026_1_09_2020_clips/TUH_026_1_09_2020_clip...,59.969547


### Video modifications

In [22]:
# Select the video modification
clip_modification = t3.WidgetMaker()
clip_modification

WidgetMaker(children=(IntText(value=0, description='Number of modifications:', style=DescriptionStyle(descript…

In [18]:
# Install ffmpeg with GPU version
!git clone https://github.com/rokibulislaam/colab-ffmpeg-cuda.git
!cp -r ./colab-ffmpeg-cuda/bin/. /usr/bin/

Cloning into 'colab-ffmpeg-cuda'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 62 (delta 17), reused 6 (delta 6), pack-reused 37
Unpacking objects: 100% (62/62), done.
Checking out files: 100% (32/32), done.


In [55]:
import os, shutil, ffmpeg
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import logging
import difflib
from pathlib import Path

from tqdm import tqdm
from IPython.display import HTML, display, update_display, clear_output
from ipywidgets import interact, interactive, Layout
from kso_utils.zooniverse_utils import auth_session
import ipywidgets as widgets

import kso_utils.db_utils as db_utils
import kso_utils.zooniverse_utils as zooniverse_utils
import kso_utils.movie_utils as movie_utils
import kso_utils.server_utils as server_utils
import kso_utils.tutorials_utils as tutorials_utils
import kso_utils.koster_utils as koster_utils
import kso_utils.project_utils as project_utils
from panoptes_client import (
    SubjectSet,
    Subject,
    Project,
    Panoptes,
)

def modify_clips(clips_to_upload_df, movie_i, modification_details, project, gpu_available = False):

    server = project.server

    # Specify the folder to host the modified clips
    if server == "SNIC":
        mod_clips_folder = "/cephyr/NOBACKUP/groups/snic2021-6-9/tmp_dir/"+"modified_"+movie_i+"_clips"
    else:
        mod_clips_folder = "modified_" + movie_i +"_clips"
    
    # Specify the path of the modified clips
    clips_to_upload_df["modif_clip_path"] = str(Path(mod_clips_folder, "modified")) + clips_to_upload_df["clip_filename"]
    
    # Remove existing modified clips
    if os.path.exists(mod_clips_folder):
        shutil.rmtree(mod_clips_folder)

    if len(modification_details.values()) > 0:
        
        # Save the modification details to include as subject metadata
        clips_to_upload_df["clip_modification_details"] = str(modification_details)
        
        # Create the folder to store the videos if not exist
        if not os.path.exists(mod_clips_folder):
            os.mkdir(mod_clips_folder)

        #### Modify the clips###
        # Read each clip and modify them (printing a progress bar) 
        for index, row in tqdm(clips_to_upload_df.iterrows(), total=clips_to_upload_df.shape[0]): 
            if not os.path.exists(row['modif_clip_path']):
                # Set up input prompt
                if gpu_available:
                  print("trying the GPU")
                  init_prompt = f"ffmpeg.input('{row['clip_path']}', hwaccel='cuda', hwaccel_output_format='cuda')"
                else:
                  init_prompt = f"ffmpeg.input('{row['clip_path']}',)"
                full_prompt = init_prompt
                # Set up modification
                for transform in modification_details.values():
                    if "filter" in transform:
                        mod_prompt = transform['filter']
                        full_prompt += mod_prompt
                # Setup output prompt
                crf_value = [transform["crf"] if "crf" in transform else None for transform in modification_details.values()]
                crf_value = [i for i in crf_value if i is not None]
                
                if len(crf_value) > 0:
                  crf_prompt = str(max([int(i) for i in crf_value]))
                  full_prompt += f".output('{row['modif_clip_path']}', crf={crf_prompt}, pix_fmt='yuv420p', vcodec='libx264')"
                else:
                  full_prompt += f".output('{row['modif_clip_path']}', crf=20, pix_fmt='yuv420p', vcodec='libx264')"
                # Run the modification
                try:
                    eval(full_prompt).run(capture_stdout=True, capture_stderr=True)
                    os.chmod(row['modif_clip_path'], 0o755)
                except ffmpeg.Error as e:
                    print('stdout:', e.stdout.decode('utf8'))
                    print('stderr:', e.stderr.decode('utf8'))
                    raise e

        print("Clips modified successfully")
        return clips_to_upload_df
    
    else:
        
        # Save the modification details to include as subject metadata
        clips_to_upload_df["modif_clip_path"] = "no_modification"
        
        return clips_to_upload_df

In [ ]:
# Modify the clips
clips_ready_df = modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True),
                              movie_i = movie_i.value,
                              modification_details = clip_modification.checks,
                              gpu_available = True,
                              project = project
                              )

In [ ]:
# Review the size of the modified clips
t3.check_clip_size(clips_df = clips_ready_df)

In [ ]:
# Compare the original and modified clips
t3.compare_clips(df = clips_ready_df)

### Set Zooniverse metadata

In [ ]:
upload_to_zoo, sitename, created_on = t3.set_zoo_metadata(db_info_dict = db_info_dict, 
                                                          df = clips_ready_df,
                                                          project = project)

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t3.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = project.Zooniverse_number)

In [ ]:
#END